# EE660 Final Project Mingjian Shi
## Airline Passenger Satisfaction Prediction 


## Best model presentation 

## 0. Preprocessing data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.semi_supervised import SelfTrainingClassifier

In [2]:
#Load the data
org_data = pd.read_csv('dataset.csv')

#one-hot encoding
org_data = pd.get_dummies(org_data, drop_first = True)

#drop "Unnamed" col
org_data.drop(["Unnamed: 0"],axis = 1, inplace = True)

#Missing data treatment 
org_data['arrival_delay_in_minutes'].fillna(org_data['departure_delay_in_minutes'], inplace = True)

#outliers removing 
data = org_data[(org_data['flight_distance'] <= 4000)]
data = org_data[(org_data['departure_delay_in_minutes'] <= 450)]
data = org_data[(org_data['arrival_delay_in_minutes'] <= 450)]

#train test split
x = data.drop('satisfaction_satisfied', axis=1)
y = data['satisfaction_satisfied']
x_train, x_test, y_train, y_test = train_test_split(x.values, y.values, test_size = 0.2, random_state = 42)

#Normalization
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

#PCA
pca = PCA(n_components = 0.85)
x_train = pca.fit_transform(x_train)
x_test = pca.fit_transform(x_test)

## 1. Supervised Learning Best Model - Random Forest

In [6]:
# Then We use the best parameters to finalize our model
RF_model=RandomForestClassifier(n_estimators=100, max_depth=50, max_features=6, bootstrap= True)
RF_model.fit(x_train, y_train)

y_train_pred = RF_model.predict(x_train)
y_test_pred = RF_model.predict(x_test)

RF_train_acc = accuracy_score(y_train, y_train_pred)
RF_test_acc = accuracy_score(y_test, y_test_pred)

print('RF Training Accuracy: ' , RF_train_acc)
print('RF Test Accuracy: ', RF_test_acc )

RF Training Accuracy:  0.9999807418249047
RF Test Accuracy:  0.9018641195501463


## 2. Semi-supervised Learning Best Model - Self Training Classifier 

In [5]:
# Form SSL data set

# Test Set
X_test = x_test
Y_test = y_test


#Original Train set with all the label
X_train = x_train
Y_train = y_train

#Decrease the train size
x_train, X_preTrain, y_train, Y_preTrain = train_test_split(X_train, Y_train, test_size = 1/3, random_state = 42)
X, X_left, Y, Y_left = train_test_split(X_preTrain, Y_preTrain, test_size = 1/2, random_state = 42)


#Use Train set to set Labeled train : Unlabeled train = 1:1
X_train_L, X_train_U, Y_train_L, Y_train_U =train_test_split(X, Y, test_size = 1/2, random_state = 42)

# Note: unlabeled examples marked with a label of -1.

#Concatenate the labeled and unlabeled data
X_train_mix = np.concatenate((X_train_L, X_train_U))

#encode -1 for unlabeled set
u_label = [-1 for _ in range(len(Y_train_U))]

#Concatenate u_label with labeled set to form the training set
Y_train_mix = np.concatenate((Y_train_L, u_label))



In [6]:
# Self-Training model
svc = SVC(probability = True, gamma = "auto")
self_training = SelfTrainingClassifier(svc)
self_training.fit(X_train_mix, Y_train_mix)

#Make prediction 
Y_pred = self_training.predict(X_test)
acc_ST = accuracy_score(Y_test, Y_pred)
print('Self Training model accuracy: ', acc_ST)

Self Training model accuracy:  0.8748266830996765
